In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Merge all the main files of the particular day into a single CSV file

# Step 1: Read all CSV files into separate DataFrames
folder_path = "buildings_by_date/Building 2/main"
file_list = os.listdir(folder_path)
dfs = []

for file_name in file_list:
    if file_name.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path, file_name))
        dfs.append(df)

# Step 2: Merge DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Step 3: Determine the minimum and maximum timestamps
min_timestamp = merged_df['Timestamp'].min()
max_timestamp = merged_df['Timestamp'].max()

# Step 4: Resample DataFrame to fill in missing timestamps
merged_df['Timestamp'] = pd.to_datetime(merged_df['Timestamp'])
merged_df.set_index('Timestamp', inplace=True)
merged_df = merged_df.asfreq('s')

# Step 5: Replace NaN values with 0
merged_df.fillna(0, inplace=True)

# Step 6: Write merged DataFrame to a new CSV file
merged_csv_path = "buildings_by_date/Building 2/building_2_main_output.csv"
merged_df.to_csv(merged_csv_path)

In [43]:
# Merge the appliance files into a single CSV file

# Define the path to the main folder containing building folders
main_folder_path = 'buildings_by_date/Building 6'

# List of appliance names to look for in each building folder
appliances = ["dryer", "kettle", "vacuum", "water_heater", "oven", "fridge", "washing_machine", "aircond"]

# Function to find all files that contain the appliance name in their filename
def find_appliance_files(folder_path, appliance_name):
    appliance_files = []
    for file in os.listdir(folder_path):
        if appliance_name in file and file.endswith('.csv'):
            appliance_files.append(os.path.join(folder_path, file))
    return appliance_files

# Dictionary to hold data for each appliance
data_frames = {}

# Loop through each appliance and create an empty DataFrame
for appliance in appliances:
    data_frames[appliance] = pd.DataFrame({'Timestamp': [], 'Active (W)': []})

# Loop through each appliance and try to find and load its data
for appliance in appliances:
    appliance_files = find_appliance_files(main_folder_path, appliance)
    if appliance_files:
        # Read all CSV files for the appliance
        dfs = []
        for file in appliance_files:
            df = pd.read_csv(file, usecols=['Timestamp', 'Active (W)']).fillna(0)
            dfs.append(df)
        data_frames[appliance] = pd.concat(dfs, ignore_index=True)

# Initialize a DataFrame to merge all appliance data
consolidated_df = pd.DataFrame()

# Loop through each appliance DataFrame and merge
for appliance, df in data_frames.items():

    # Rename 'Active (W)' to the appliance name for clarity
    df = df.rename(columns={'Active (W)': appliance})
    if consolidated_df.empty:
        consolidated_df = df
    else:
        # Merge on 'Timestamp', ensuring to use an outer join to keep all timestamps
        consolidated_df = pd.merge(consolidated_df, df, on='Timestamp', how='outer')

    # Show head of each appliance file
    print(consolidated_df.head())

consolidated_df.fillna(0, inplace=True)

# Save the consolidated DataFrame to a new CSV file
consolidated_csv_path = os.path.join(main_folder_path, "appliances_by_column.csv")
consolidated_df.to_csv(consolidated_csv_path, index=False)
print(f"Consolidated CSV created for {main_folder_path}")

             Timestamp  dryer
0  2022-11-10 11:53:42   0.17
1  2022-11-10 11:53:43   0.15
2  2022-11-10 11:53:44   0.13
3  2022-11-10 11:53:45   0.14
4  2022-11-10 11:53:46   0.16
             Timestamp  dryer  kettle
0  2022-11-10 11:53:42   0.17     NaN
1  2022-11-10 11:53:43   0.15     NaN
2  2022-11-10 11:53:44   0.13     NaN
3  2022-11-10 11:53:45   0.14     NaN
4  2022-11-10 11:53:46   0.16     NaN
             Timestamp  dryer  kettle  vacuum
0  2022-11-10 11:53:42   0.17     NaN     NaN
1  2022-11-10 11:53:43   0.15     NaN     NaN
2  2022-11-10 11:53:44   0.13     NaN     NaN
3  2022-11-10 11:53:45   0.14     NaN     NaN
4  2022-11-10 11:53:46   0.16     NaN     NaN
             Timestamp  dryer  kettle  vacuum  water_heater
0  2022-11-10 11:53:42   0.17     NaN     NaN           NaN
1  2022-11-10 11:53:43   0.15     NaN     NaN           NaN
2  2022-11-10 11:53:44   0.13     NaN     NaN           NaN
3  2022-11-10 11:53:45   0.14     NaN     NaN           NaN
4  2022-11-10 11

In [36]:
# Merge the main and appliance CSV files into a single CSV file

# Path to your main CSV file
main_csv_path = 'buildings_by_date/Building 5/building_5_main_output.csv'

# Load the main CSV file
main_df = pd.read_csv(main_csv_path)
print(main_df.head())

# Define path to the appliance CSV file
appliance_csv_path = 'buildings_by_date/Building 5/appliances_by_column.csv'

# Load the appliance CSV file
appliances_df = pd.read_csv(appliance_csv_path)
print(appliances_df.head())

# Merge the main and appliance DataFrames on the 'Timestamp' column
merged_df = pd.merge(main_df, appliances_df, on='Timestamp', how='outer')
print(merged_df.head())

# Fill any NaN values with 0
merged_df.fillna(0, inplace=True)

# Save the merged DataFrame to a new CSV file
merged_csv_path = 'mimos_final_buildings/1_sec/Building_5.csv'
merged_df.to_csv(merged_csv_path, index=False)

             Timestamp  Apparent (VA)  Active (W)
0  2022-11-08 10:39:23          23.97       13.50
1  2022-11-08 10:39:24          23.72       13.20
2  2022-11-08 10:39:25          23.55       13.00
3  2022-11-08 10:39:26          24.05       13.35
4  2022-11-08 10:39:27          23.69       13.22
             Timestamp  kettle  vacuum  water_heater  oven  fridge  \
0  2022-11-08 10:39:23     0.0     0.0           0.0   0.0     0.0   
1  2022-11-08 10:39:24     0.0     0.0           0.0   0.0     0.0   
2  2022-11-08 10:39:25     0.0     0.0           0.0   0.0     0.0   
3  2022-11-08 10:39:26     0.0     0.0           0.0   0.0     0.0   
4  2022-11-08 10:39:27     0.0     0.0           0.0   0.0     0.0   

   washing_machine  dryer  aircond  
0            11.07    0.0      0.0  
1            11.32    0.0      0.0  
2            11.33    0.0      0.0  
3            11.30    0.0      0.0  
4            11.23    0.0      0.0  
             Timestamp  Apparent (VA)  Active (W)  kettle

In [19]:
def resample_csv(input_path, output_path, sampling_period='6s'):
    """Resample the dataset to a new frequency.

    Parameters:
    input_path (str): Path to the input CSV file.
    output_path (str): Path to save the resampled CSV file.
    sampling_period (str): New sampling frequency (default is '6s' for 6 seconds).
    """
    # Load the dataset
    df = pd.read_csv(input_path, index_col=0, parse_dates=True)

    # Resample the dataset
    resampled_df = df.resample(sampling_period).mean()

    # Drop rows where all columns (except for the index) are NaN
    resampled_df = resampled_df.dropna(how='all')

    # Save the resampled dataset
    resampled_df.to_csv(output_path)

file_names = [
    "Building_1.csv",
    "Building_2.csv",
    "Building_3.csv",
    "Building_4.csv",
    "Building_5.csv",
    "Building_6.csv",
    "Building_7.csv"
]

input_directory = "mimos_final_buildings/1_sec"
output_directory = "mimos_final_buildings/3_sec"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for file_name in file_names:
    input_path = os.path.join(input_directory, file_name)
    output_path = os.path.join(output_directory, file_name)
    print(f"Resampling {file_name}...")
    resample_csv(input_path, output_path, '3s')
    print(f"Saved resampled file to {output_path}")

Resampling Building_1.csv...
Saved resampled file to mimos_final_buildings/3_sec\Building_1.csv
Resampling Building_2.csv...
Saved resampled file to mimos_final_buildings/3_sec\Building_2.csv
Resampling Building_3.csv...
Saved resampled file to mimos_final_buildings/3_sec\Building_3.csv
Resampling Building_4.csv...
Saved resampled file to mimos_final_buildings/3_sec\Building_4.csv
Resampling Building_5.csv...
Saved resampled file to mimos_final_buildings/3_sec\Building_5.csv
Resampling Building_6.csv...
Saved resampled file to mimos_final_buildings/3_sec\Building_6.csv
Resampling Building_7.csv...
Saved resampled file to mimos_final_buildings/3_sec\Building_7.csv


In [ ]:
# Define the folder containing the CSV files
folder_path = 'path_to_your_folder'

# Define the desired column order
desired_columns = ['Timestamp', 'UNIX', 'Active (W)', 'Apparent (VA)', 'fridge', 'air conditioner', 
                   'washing machine', 'tumble dryer', 'kettle', 'vacuum cleaner', 
                   'electric water heating appliance', 'oven']

# Iterate through each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)
        
        # Rearrange columns according to desired order
        df = df[desired_columns]
        
        # Save the modified DataFrame back to the CSV file
        df.to_csv(file_path, index=False)
        
        print(f"Columns rearranged for file: {filename}")

print("All files processed successfully.")

In [ ]:
# Check for Duplicate Timestamps

# Step 1: Read CSV files into DataFrames
folder_path = "buildings_by_date/Building 5/main"
file_names = os.listdir(folder_path)
dfs = []

for file_name in file_names:
    if file_name.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path, file_name))
        dfs.append(df)

# Step 2: Merge DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Step 3: Check for duplicate timestamps
duplicates = merged_df.duplicated(subset=['Timestamp'], keep=False)
if duplicates.any():
    print("Warning: Duplicate timestamps found.")
    duplicate_rows = merged_df[duplicates]
    duplicate_timestamps = duplicate_rows['Timestamp'].unique()
    print("Duplicate Timestamps:")
    for timestamp in duplicate_timestamps:
        print(timestamp)

In [47]:
# Dealing with duplicates by summing up the values

# Read the two CSV files into DataFrames
df1 = pd.read_csv('buildings_by_date/Building 4/main/2022-11-07-17_15_17_main_output.csv')
df2 = pd.read_csv('buildings_by_date/Building 4/main/2022-11-07-17_28_46_main_output.csv')

# Merge the two DataFrames on the Timestamp column
merged_df = pd.merge(df1, df2, on='Timestamp')

# Add up the Active (W) and Apparent (W) values
merged_df['Active (W)'] = merged_df['Active (W)_x'] + merged_df['Active (W)_y']
merged_df['Apparent (VA)'] = merged_df['Apparent (VA)_x'] + merged_df['Apparent (VA)_y']

# Drop the original Active (W) and Apparent (W) columns from the merged DataFrame
merged_df.drop(['Active (W)_x', 'Active (W)_y', 'Apparent (VA)_x', 'Apparent (VA)_y'], axis=1, inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('buildings_by_date/Building 4/main/no_more_duplicate.csv', index=False)

In [29]:
# Change timestamp into correct format

# Read the CSV file into a DataFrame
df = pd.read_csv('buildings_by_date/Building 6/main/2022-11-09-10_00_06_main_output.csv')

# Convert the Timestamp column to datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Convert the Timestamp column to the desired format
df['Timestamp'] = df['Timestamp'].dt.strftime('%m/%d/%Y %H:%M:%S')

# Save the DataFrame back to a CSV file
df.to_csv('output_file.csv', index=False)